In [1]:
from openpyxl import load_workbook
import random
import io

workbook = load_workbook(u'lena_for_cross_domain.xlsx')
booksheet = workbook.active
rows = booksheet.rows
columns = booksheet.columns

line_list = []
for row in rows:
    line = [col.value for col in row]
    line_list.append(line)
    
dict = {}
utters = []
utter_subintents = []
for line in line_list[1:]:
    utters.append(line[1])
    utter_subintents.append(line[2])
    supintent = line[2]
    if supintent not in dict.keys():
        dict[supintent] = []
    dict[supintent].append(line[1])
    
def movestopwords(sentence):
    counts = {}
    for sen in sentence:
        for word in sen.split():
            if len(word) != 1:
                counts[word] = counts.get(word,0) + 1
    return counts

In [4]:
dict_count = {} 
supintent_f_in_cases = []
subintents = []
for k,v in dict.items():
    subintents.append(k)
    dict_count[k] = movestopwords(v)
train_list = []
for i in range(len(utters)):
    sort_ = {}
    d = dict_count[utter_subintents[i]] # utter对应的label下的utter集合的词频统计
    for k in subintents: # k个label
        sort_i = 0
        for ki in k.split():
            if ki in dict_count[utter_subintents[i]].keys():
                sort_i += dict_count[utter_subintents[i]][ki]
            else:
                sort_i += 0
        sort_[k] = sort_i
    c_order = sorted(sort_.items(),key=lambda x:x[1],reverse=True)
    c_order = c_order[:10]
    # negative full
    for negative_subintent in c_order:
        neg_text = "0\t" + utters[i] + "\t" + negative_subintent[0]
        train_list.append(neg_text)
    text = "1\t" + utters[i] + "\t" + utter_subintents[i]
    train_list.append(text)
random.shuffle(train_list)
fdata = open("./esim/lena/data_test_10.txt", 'w', encoding="utf-8")
for text in train_list:
        fdata.write(text + "\n")
fdata.close()
print("Finish")

Finish


In [6]:
train_list = []
supintent_names = list(dict.keys())
for supintent in dict.keys():
    supintent_names.remove(supintent)
    for case in dict[supintent][:1000]:
        # negative full
        for negative_intent in supintent_names:
            neg_text = "0\t" + case + "\t" + negative_intent
            train_list.append(neg_text)
        text = "1\t" + case + "\t" + supintent
        train_list.append(text)
    supintent_names.append(supintent)
random.shuffle(train_list)
fdata = open("./esim/lena/data_test.txt", 'w', encoding="utf-8")
for text in train_list:
        fdata.write(text + "\n")
fdata.close()
print("Finish")

Finish
